# Portfolio A: Text Classification
**Build a content moderation pipeline for an AI agent social network**

You've been hired to build a content classifier for *Moltbook* — a leaked social network where AI agents post, argue, and share memes. Your job: classify posts into 9 content categories and identify patterns in how agents communicate.

**Dataset**: Moltbook (44K AI agent posts, 9 categories)
**Your goal**: Build and compare at least 2 classification approaches, evaluate rigorously, and explain what works and why.

### Deliverables
- Working classification pipeline with at least 2 approaches
- Evaluation: confusion matrix, per-class F1, macro F1
- Error analysis: 5+ misclassified examples with explanations
- Brief model card (data, method, limits, failure modes)

**Estimated time**: Sprint 1 (55 min) + Sprint 2 (90 min)

## Setup

In [ ]:
!pip install -q datasets sentence-transformers openai scikit-learn matplotlib seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

## 1. Load & Explore the Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("TrustAIRLab/Moltbook", "posts", split="train")
df = dataset.to_pandas()

# Flatten nested post column
df["title"] = df["post"].apply(lambda x: x.get("title", "") if isinstance(x, dict) else "")
df["content"] = df["post"].apply(lambda x: x.get("content", "") if isinstance(x, dict) else "")
df["title"] = df["title"].fillna("").astype(str)
df["content"] = df["content"].fillna("").astype(str)
df["text"] = (df["title"].str.strip() + " . " + df["content"].str.strip()).str.strip()

# Use topic_label as classification target
df = df[df["text"].str.len() > 10].reset_index(drop=True)
print(f"Dataset: {len(df)} posts")
print(f"\nLabel distribution:")
print(df["topic_label"].value_counts())

In [ ]:
# Quick look at the data
for label in df["topic_label"].unique()[:3]:
    example = df[df["topic_label"] == label].iloc[0]
    print(f"\n{'='*60}")
    print(f"Label: {label}")
    print(f"Text: {example['text'][:200]}...")

## 2. Preprocessing & Train/Test Split

In [ ]:
def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"https?://\S+", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

df["text_clean"] = df["text"].apply(clean_text)

# Subsample for speed (use full dataset if time allows)
df_work = df.sample(5000, random_state=42).reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(
    df_work["text_clean"],
    df_work["topic_label"],
    test_size=0.25,
    stratify=df_work["topic_label"],
    random_state=42,
)
print(f"Train: {len(X_train)}, Test: {len(X_test)}")

## 3. Baseline: TF-IDF + Logistic Regression
This is your floor — every other approach should beat this.

In [ ]:
pipe_lr = Pipeline([
    ("tfidf", TfidfVectorizer(stop_words="english", ngram_range=(1, 2), max_features=10_000)),
    ("clf", LogisticRegression(max_iter=1000, random_state=42)),
])

pipe_lr.fit(X_train, y_train)
y_pred_baseline = pipe_lr.predict(X_test)

print(f"TF-IDF + LR Accuracy: {accuracy_score(y_test, y_pred_baseline):.4f}")
print(classification_report(y_test, y_pred_baseline))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_baseline, ax=ax, cmap="Blues", xticks_rotation=45)
ax.set_title("Baseline: TF-IDF + Logistic Regression")
plt.tight_layout()
plt.show()

## 4. Your Turn: Add a Second Approach

Choose one (or more):
- **Sentence embeddings** (from NB02): Encode with `all-MiniLM-L6-v2`, train a classifier on embeddings
- **LLM zero-shot** (from NB03): Use Groq API to classify without training data
- **SetFit few-shot** (from NB05): Train on just 8-32 examples per class

In [ ]:
# YOUR CODE HERE — Approach 2
# Example: Sentence Embeddings
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer("all-MiniLM-L6-v2")
# X_train_emb = model.encode(X_train.tolist(), show_progress_bar=True)
# X_test_emb = model.encode(X_test.tolist(), show_progress_bar=True)
# clf = LogisticRegression(max_iter=1000, random_state=42)
# clf.fit(X_train_emb, y_train)
# y_pred_emb = clf.predict(X_test_emb)
# print(f"SBERT + LR Accuracy: {accuracy_score(y_test, y_pred_emb):.4f}")

## 5. Compare Results

In [ ]:
# YOUR CODE HERE — fill in your results
results = {
    "TF-IDF + LR": accuracy_score(y_test, y_pred_baseline),
    # "SBERT + LR": accuracy_score(y_test, y_pred_emb),
    # "LLM Zero-shot": ...,
}

results_df = pd.DataFrame(list(results.items()), columns=["Method", "Accuracy"])
results_df = results_df.sort_values("Accuracy", ascending=False)
print(results_df.to_string(index=False))

## 6. Error Analysis
Look at the hardest examples — where does each model fail?

In [ ]:
proba = pipe_lr.predict_proba(X_test)
error_df = pd.DataFrame({
    "text": X_test.values,
    "true_label": y_test.values,
    "predicted": y_pred_baseline,
    "confidence": proba.max(axis=1),
})
errors = error_df[error_df["true_label"] != error_df["predicted"]].sort_values("confidence", ascending=False)
print(f"Misclassified: {len(errors)}/{len(error_df)} ({len(errors)/len(error_df)*100:.1f}%)\n")

for i, row in errors.head(5).iterrows():
    print(f"True: {row['true_label']} \u2192 Predicted: {row['predicted']} (conf: {row['confidence']:.2f})")
    print(f"  {row['text'][:150]}...")
    print()

## 7. Model Card

Fill this in when you're done:

| Field | Value |
|-------|-------|
| **Task** | 9-class content classification |
| **Dataset** | Moltbook (N=5000 sample) |
| **Best method** | _your best approach_ |
| **Macro F1** | _score_ |
| **Key strength** | _what it gets right_ |
| **Key weakness** | _what it gets wrong_ |
| **Failure mode** | _describe a systematic error pattern_ |
| **Improvement idea** | _what you'd try next_ |